In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg_graph import (
    HeisenbergModel,
    get_n_steps,
    get_graph,
    get_positions,
    get_initial_layout,
    get_prob0,
    extract_probs,
)
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import mthree
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
# Option1: Use IBM Quantum backend.
# Set up the Qiskit Runtime service (this is a one-time setup)
# QiskitRuntimeService.save_account(
#     token="YOUR_API_TOKEN",
#     channel="ibm_quantum",
# )

# Load saved credentials
service = QiskitRuntimeService()
# backend_qpu = service.least_busy(simulator=False, interactional=True)
backend_qpu = service.backend("ibm_marrakesh")

# Option2: Use local AerSimulator as the backend.
backend_sim = AerSimulator()

noise_model = NoiseModel.from_backend(backend_qpu)
backend_sim_noisy = AerSimulator(noise_model=noise_model)

print(f"Using backend QPU: {backend_qpu}")
print(f"Using backend simulator: {backend_sim}")
print(f"Using backend noisy simulator: {backend_sim_noisy}")

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend simulator: AerSimulator('aer_simulator')
Using backend noisy simulator: AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['id', 'measure', 'reset', 'cz', 'sx', 'x']>)


In [5]:
# Only for CP1
%cd fourier_learning_ibm/

/home/jovyan/fourier_learning_ibm


In [ ]:
# Load parameters
# current_time can be found in data_generate.ipynb

CURRENT_TIME = "2024-12-02T09:24Z"

path = f"./data/{CURRENT_TIME}"

# Load the graphs as a binary file
with open(f"{path}/params_object.pkl", "rb") as f:
    params_object = pickle.load(f)

graphs = params_object["graphs"]
all_Js = params_object["Js"]
all_expected_values = params_object["expected_values"]

# Load the parameters as a JSON file
with open(f"{path}/params_text.json", "r") as f:
    params = json.load(f)

n_samples = params["n_samples"]
n_qubits = params["n_qubits"]
graph_type = params["graph_type"]
backend_qpu_name = params["backend_qpu_name"]
beta = params["beta"]
C = params["C"]
n_features = params["n_features"]
times = params["times"]

for key, value in params.items():
    print(f"{key}: {value}")

# Create Fourier feature

## Trotter simulation (Exact)

In [7]:
# Exact simulation
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# Generate the dataset with Fourier features
for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    heisenberg_exact = HeisenbergModel(n_qubits, graphs[i])
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        prob_phase0 = heisenberg_exact.exact_simulation(times[k], phase=0)
        prob_phase1 = heisenberg_exact.exact_simulation(times[k], phase=1)
        prob_phase2 = heisenberg_exact.exact_simulation(times[k], phase=2)
        prob_phase3 = heisenberg_exact.exact_simulation(times[k], phase=3)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_exact_df = pd.DataFrame(data, columns=columns)
display(data_exact_df)

# Save the exact data
data_exact_df.to_json(f"{path}/data_exact_df.json")
with open(f"{path}/probs_phase0_exact.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1_exact.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2_exact.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3_exact.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/108


/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


Calculating features for sample 1/108
Calculating features for sample 2/108
Calculating features for sample 3/108
Calculating features for sample 4/108
Calculating features for sample 5/108
Calculating features for sample 6/108
Calculating features for sample 7/108
Calculating features for sample 8/108
Calculating features for sample 9/108
Calculating features for sample 10/108
Calculating features for sample 11/108
Calculating features for sample 12/108
Calculating features for sample 13/108
Calculating features for sample 14/108
Calculating features for sample 15/108
Calculating features for sample 16/108
Calculating features for sample 17/108
Calculating features for sample 18/108
Calculating features for sample 19/108
Calculating features for sample 20/108
Calculating features for sample 21/108
Calculating features for sample 22/108
Calculating features for sample 23/108
Calculating features for sample 24/108
Calculating features for sample 25/108
Calculating features for sample 26

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,expected_value
0,0,1.0,0.996941,-0.021182,0.987812,-0.041879,0.972755,-0.061616,0.952003,-0.079948,2.349069
1,1,1.0,0.994417,-0.084961,0.977772,-0.168494,0.950368,-0.249200,0.912704,-0.325731,0.801446
2,2,1.0,0.998612,-0.022648,0.994457,-0.045169,0.987562,-0.067437,0.977969,-0.089325,1.401454
3,3,1.0,0.999344,-0.008507,0.997379,-0.016955,0.994120,-0.025286,0.989588,-0.033445,1.179205
4,4,1.0,0.997933,-0.025646,0.991765,-0.050839,0.981590,-0.075134,0.967564,-0.098100,1.121292
...,...,...,...,...,...,...,...,...,...,...,...
103,103,1.0,0.997535,0.036264,0.990166,0.072380,0.977970,0.108198,0.961077,0.143564,5.071432
104,104,1.0,0.999175,0.019264,0.996705,0.038472,0.992609,0.057569,0.986913,0.076502,2.536326
105,105,1.0,0.997997,0.033793,0.992007,0.067440,0.982089,0.100798,0.968341,0.133718,4.458616
106,106,1.0,0.996319,0.048269,0.985321,0.096200,0.967140,0.143451,0.941994,0.189683,7.820448


## Trotter simulation (Simulator, Noise free)

In [8]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    heisenberg_sim = HeisenbergModel(n_qubits, graphs[i], backend=backend_sim)

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=0)
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=1)
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=2)
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=3)
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/108
Creating circuits for sample 1/108
Creating circuits for sample 2/108
Creating circuits for sample 3/108
Creating circuits for sample 4/108
Creating circuits for sample 5/108
Creating circuits for sample 6/108
Creating circuits for sample 7/108
Creating circuits for sample 8/108
Creating circuits for sample 9/108
Creating circuits for sample 10/108
Creating circuits for sample 11/108
Creating circuits for sample 12/108
Creating circuits for sample 13/108
Creating circuits for sample 14/108
Creating circuits for sample 15/108
Creating circuits for sample 16/108
Creating circuits for sample 17/108
Creating circuits for sample 18/108
Creating circuits for sample 19/108
Creating circuits for sample 20/108
Creating circuits for sample 21/108
Creating circuits for sample 22/108
Creating circuits for sample 23/108
Creating circuits for sample 24/108
Creating circuits for sample 25/108
Creating circuits for sample 26/108
Creating circuits for sample 27/108
Cr

In [9]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/108
Running circuits for sample 1/108
Running circuits for sample 2/108
Running circuits for sample 3/108
Running circuits for sample 4/108
Running circuits for sample 5/108
Running circuits for sample 6/108
Running circuits for sample 7/108
Running circuits for sample 8/108
Running circuits for sample 9/108
Running circuits for sample 10/108
Running circuits for sample 11/108
Running circuits for sample 12/108
Running circuits for sample 13/108
Running circuits for sample 14/108
Running circuits for sample 15/108
Running circuits for sample 16/108
Running circuits for sample 17/108
Running circuits for sample 18/108
Running circuits for sample 19/108
Running circuits for sample 20/108
Running circuits for sample 21/108
Running circuits for sample 22/108
Running circuits for sample 23/108
Running circuits for sample 24/108
Running circuits for sample 25/108
Running circuits for sample 26/108
Running circuits for sample 27/108
Running circuits for sample 28

In [10]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,958c7775-0bf5-4e81-aca0-e01dbead4081,0,DONE
1,706c8fc1-4be7-4372-a8fc-8d44d1fa29ca,1,DONE
2,418c5ef1-8848-4ec9-bf1d-399e8645e451,2,DONE
3,a4436cf6-9c18-410d-9350-b1bbca3d9481,3,DONE
4,8b1d7060-2726-474d-a8bf-64b342162b93,4,DONE
...,...,...,...
103,6f06463d-a6c1-43bf-881a-c3e7dffac8cb,103,DONE
104,4f543bad-c4e2-4cc9-85a8-06407872abf2,104,DONE
105,c6557223-65b1-4290-a884-10d648f63e9d,105,DONE
106,eb2d20eb-c81a-44d6-b722-8af26be67ec1,106,DONE


In [11]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_df = pd.DataFrame(data_sim, columns=columns)
display(data_sim_df)

# Save the simulation data
data_sim_df.to_json(f"{path}/data_sim_df.json")
with open(f"{path}/probs_phase0_sim.json", "w") as f:
    json.dump(probs_phase0_sim, f)

with open(f"{path}/probs_phase1_sim.json", "w") as f:
    json.dump(probs_phase1_sim, f)

with open(f"{path}/probs_phase2_sim.json", "w") as f:
    json.dump(probs_phase2_sim, f)

with open(f"{path}/probs_phase3_sim.json", "w") as f:
    json.dump(probs_phase3_sim, f)

 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No co

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,expected_value
0,0,1.0,0.998071,-0.031530,0.993593,-0.049453,0.976799,-0.038686,0.953132,-0.097080,2.349069
1,1,1.0,0.993240,-0.082332,0.978283,-0.169554,0.948136,-0.262947,0.915999,-0.299850,0.801446
2,2,1.0,0.999012,-0.022966,0.997013,-0.057649,0.989113,-0.096227,0.980293,-0.087920,1.401454
3,3,1.0,0.997027,-0.011013,0.994713,-0.028791,0.996991,-0.007631,0.993185,-0.047752,1.179205
4,4,1.0,1.001722,-0.042657,0.990228,-0.062092,0.979798,-0.094583,0.967027,-0.134063,1.121292
...,...,...,...,...,...,...,...,...,...,...,...
103,103,1.0,1.000429,0.008386,1.012120,0.015201,0.973590,0.120786,0.963059,0.138361,5.071432
104,104,1.0,0.998091,0.045276,0.995238,0.014151,0.992070,0.027766,0.986309,0.070399,2.536326
105,105,1.0,0.996856,0.051179,0.994956,0.065169,0.976323,0.118542,0.965291,0.113892,4.458616
106,106,1.0,0.997076,-0.002161,0.983704,0.099363,0.953438,0.115426,0.939879,0.206677,7.820448


## Trotter simulation (Simulator, Noisy)

In [12]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]
    heisenberg_sim_noisy = HeisenbergModel(
        n_qubits, graphs[i], backend=backend_sim_noisy
    )

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=0
            )
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=1
            )
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=2
            )
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=3
            )
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/108
Creating circuits for sample 1/108
Creating circuits for sample 2/108
Creating circuits for sample 3/108
Creating circuits for sample 4/108
Creating circuits for sample 5/108
Creating circuits for sample 6/108
Creating circuits for sample 7/108
Creating circuits for sample 8/108
Creating circuits for sample 9/108
Creating circuits for sample 10/108
Creating circuits for sample 11/108
Creating circuits for sample 12/108
Creating circuits for sample 13/108
Creating circuits for sample 14/108
Creating circuits for sample 15/108
Creating circuits for sample 16/108
Creating circuits for sample 17/108
Creating circuits for sample 18/108
Creating circuits for sample 19/108
Creating circuits for sample 20/108
Creating circuits for sample 21/108
Creating circuits for sample 22/108
Creating circuits for sample 23/108
Creating circuits for sample 24/108
Creating circuits for sample 25/108
Creating circuits for sample 26/108
Creating circuits for sample 27/108
Cr

In [13]:
# Run jobs in batch
jobs = []  # For AerSimulator, we can't use job ids.
with Batch(backend=backend_sim_noisy):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/108
Running circuits for sample 1/108
Running circuits for sample 2/108
Running circuits for sample 3/108
Running circuits for sample 4/108
Running circuits for sample 5/108
Running circuits for sample 6/108
Running circuits for sample 7/108
Running circuits for sample 8/108
Running circuits for sample 9/108
Running circuits for sample 10/108
Running circuits for sample 11/108
Running circuits for sample 12/108
Running circuits for sample 13/108
Running circuits for sample 14/108
Running circuits for sample 15/108
Running circuits for sample 16/108
Running circuits for sample 17/108
Running circuits for sample 18/108
Running circuits for sample 19/108
Running circuits for sample 20/108
Running circuits for sample 21/108
Running circuits for sample 22/108
Running circuits for sample 23/108
Running circuits for sample 24/108
Running circuits for sample 25/108
Running circuits for sample 26/108
Running circuits for sample 27/108
Running circuits for sample 28

In [14]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,c3688294-2a60-4a60-827b-bd556e4f9498,0,DONE
1,174dafc7-f4ca-4c82-807e-d45cbf506acf,1,DONE
2,4428612d-ac94-4564-a839-3b67211d6da5,2,DONE
3,6159b0f9-8cff-49aa-802f-71605ce7bfe9,3,DONE
4,ff945ad2-0f51-4759-bae2-d9c35b08146c,4,DONE
...,...,...,...
103,9c2633e6-4699-4f5b-adff-3b29d532a485,103,DONE
104,df2cf7d1-bc44-4432-8713-ccf3c174732e,104,DONE
105,495d4abf-5346-4450-bd9c-d12e8f7e3317,105,DONE
106,bfd9ccbf-9dd4-4eb8-9a6d-727cb55dab8c,106,DONE


In [15]:
# Post-process
data_sim_noisy = []
probs_phase0_sim_noisy = {}
probs_phase1_sim_noisy = {}
probs_phase2_sim_noisy = {}
probs_phase3_sim_noisy = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim_noisy[f"sample{i}"] = {}
    probs_phase1_sim_noisy[f"sample{i}"] = {}
    probs_phase2_sim_noisy[f"sample{i}"] = {}
    probs_phase3_sim_noisy[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )
        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim_noisy.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_noisy_df = pd.DataFrame(data_sim_noisy, columns=columns)
display(data_sim_noisy_df)

# Save the simulation data
data_sim_noisy_df.to_json(f"{path}/data_sim_noisy_df.json")
with open(f"{path}/probs_phase0_sim_noisy.json", "w") as f:
    json.dump(probs_phase0_sim_noisy, f)

with open(f"{path}/probs_phase1_sim_noisy.json", "w") as f:
    json.dump(probs_phase1_sim_noisy, f)

with open(f"{path}/probs_phase2_sim_noisy.json", "w") as f:
    json.dump(probs_phase2_sim_noisy, f)

with open(f"{path}/probs_phase3_sim_noisy.json", "w") as f:
    json.dump(probs_phase3_sim_noisy, f)

 > No counts for |0...0> state
 > No counts for |0...0> state


,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,expected_value
0,0,0.885742,0.675436,0.012090,0.686971,-0.045518,0.687695,-0.046915,0.667627,-0.022341,2.349069
1,1,0.861328,0.702509,-0.029831,0.686715,-0.111224,0.688187,-0.178733,0.621642,-0.227991,0.801446
2,2,0.884766,0.688464,-0.025739,0.693335,-0.024139,0.666954,-0.025426,0.701048,-0.061965,1.401454
3,3,0.868164,0.665340,0.012181,0.681133,0.002345,0.682812,-0.032737,0.679584,-0.024535,1.179205
4,4,0.875977,0.703007,-0.005446,0.682686,0.004945,0.661330,-0.036751,0.693755,-0.047973,1.121292
...,...,...,...,...,...,...,...,...,...,...,...
103,103,0.875000,0.685028,0.013361,0.693212,0.056260,0.724481,0.045077,0.679823,0.125315,5.071432
104,104,0.875000,0.734643,0.034005,0.692602,0.018455,0.684820,0.029899,0.698256,0.075085,2.536326
105,105,0.860352,0.683857,0.032201,0.682067,0.059471,0.692417,0.073337,0.668398,0.058451,4.458616
106,106,0.883789,0.705451,0.059185,0.690258,0.061916,0.655547,0.115982,0.648167,0.141492,7.820448
